In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch
from utils import dataset_gen, minmaxscaler
from model import GNN
import pandas as pd
import os
import seaborn as sns
from torch_geometric.loader import DataLoader

device = torch.device('cpu')

In [3]:
indicators = ['WSS','OSI']#

sub_ind = [indicators[0]]

task_type = 'interp'
param_type = 'layers'
conv_type = 'gtr'
input_size = 7 # dataset features: Time, Press_SA, Press_abd, FlowRate, coord(x), coord(y), coord(z)
hidden_size = 32
layers_list = [2,3,4,5]
perc = 75
output_size = len(sub_ind)
dataset = dataset_gen(perc,sub_ind)
num_nodes = dataset[0].num_nodes
T = len(dataset)
loader = DataLoader(dataset, batch_size = 1, shuffle=False)

y = torch.zeros((T,num_nodes))
for i, data in enumerate(loader):
    y[i,:] = data.y.squeeze()
t = torch.linspace(0,1,T)

for num_layers in layers_list:
    for it in range(5):

    
        pred = torch.zeros((T,num_nodes))
        task_type = 'interp'
        log_name = '_'.join([conv_type,'hid', str(hidden_size),'nl',str(num_layers), 'loss'])
        log_path = 'interp/' + '_'.join(['layers']+sub_ind)

        # model = GNN(input_size,hidden_size,output_size,num_layers,conv_type=conv_type,device=device)

        # model.load_state_dict(torch.load(log_path+'/'+log_name + str(it) + '.pt', map_location=torch.device('cpu')))

        

        # for i, data in enumerate(loader):
            
        #     pred[i,:] = model(data.x, data.edge_index).squeeze()
        
        # model_folder = '_'.join(['store', param_type, task_type, sub_ind[0]])+'/'
        # if not os.path.exists(model_folder):
        #     os.makedirs(model_folder)
        # model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])
        
        # torch.save(pred,model_folder+model_name)




In [5]:
model_folder = '_'.join(['store', param_type, task_type, sub_ind[0]])+'/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])

In [13]:
nodes = [208,305,400,800,852]
data = []
for i, num_layers in enumerate(layers_list):
    for it in range(5):       
        model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])
        for node in nodes:
            for j in range(500):
                data.append({'num_layer': num_layers, 'it': it, 'node': node, 't':float(t[j]), 'pred': float(torch.load(model_folder+model_name).detach().numpy()[j,node])})
            #pred_plot[i,j,:,:] = torch.load('_'.join(['pred',indicators[0],str(num_layers),str(j)])).detach()
df = pd.DataFrame.from_records(data)

In [15]:
df[df['num_layer']==3][df['node']==node][['t','pred','it']]

/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/3777731001.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['num_layer']==3][df['node']==node][['t','pred','it']]


,t,pred,it
14500,0.000000,0.007152,0
14501,0.002004,0.007223,0
14502,0.004008,0.007320,0
14503,0.006012,0.007444,0
14504,0.008016,0.007606,0
...,...,...,...
24995,0.991984,0.008159,4
24996,0.993988,0.008592,4
24997,0.995992,0.009106,4
24998,0.997996,0.009737,4


In [28]:

for node in nodes:
    fig, axes = plt.subplots(1,4, sharey=True, figsize=(16, 8) )
    axes[0].set_ylabel(sub_ind[0], fontsize = '16')
    for i,l in enumerate(layers_list):
        red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
        axes[i].set_xlabel('t', fontsize = '16')
        axes[i].tick_params(axis = 'both', which='minor', labelsize= 12)
        sns.lineplot(data = red, x="t", y="pred", ax=axes[i], label = 'Prediction')
        sns.lineplot(x = t, y = y[:,node].detach(),ax=axes[i], label = 'Ground Truth')
        axes[i].legend(fontsize = '14', loc = 'upper right')

    os.makedirs('seaborn/pred_layers_interp/', exist_ok=True)
    plt.savefig('seaborn/pred_layers_interp/pred'+str(node)+'.pdf')
    plt.close()

/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Ser

TAWSS computation

In [33]:
errors = []
T = len(dataset)
dt = 1/T
data = minmaxscaler(np.load(str(perc)+'Percent.npz')['WSS'])
data[0,:] = data[0,:]/2
data[-1,:] = data[-1,:]/2
TAWSS = dt * np.sum(np.abs(data), axis = 0)
for it in range(5):
    for num_layers in layers_list:
        model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])
        
        pred = torch.load(model_folder+model_name).detach().numpy()
        pred[0,:] /= 2
        pred[-1,:] /=2
        TAWSSp = dt * np.sum(np.abs(pred[:,:]), axis = 0)
        error = np.linalg.norm(TAWSS-TAWSSp,ord= np.inf)/np.linalg.norm(TAWSS, ord = np.inf)
        errors.append({'layer_num':num_layers, 'it':it, 'error': error})
db = pd.DataFrame.from_records(errors)

In [35]:
db.groupby('layer_num').describe()

it                                         error            \
          count mean       std  min  25%  50%  75%  max count      mean   
layer_num                                                                 
2           5.0  2.0  1.581139  0.0  1.0  2.0  3.0  4.0   5.0  0.364210   
3           5.0  2.0  1.581139  0.0  1.0  2.0  3.0  4.0   5.0  0.422534   
4           5.0  2.0  1.581139  0.0  1.0  2.0  3.0  4.0   5.0  0.323707   
5           5.0  2.0  1.581139  0.0  1.0  2.0  3.0  4.0   5.0  0.403281   

                                                                       
                std       min       25%       50%       75%       max  
layer_num                                                              
2          0.050327  0.330999  0.340481  0.343409  0.353027  0.453135  
3          0.042029  0.352242  0.420146  0.431585  0.451797  0.456901  
4          0.064260  0.259841  0.292855  0.311060  0.324650  0.430129  
5          0.068852  0.340993  0.343070  0.391693  0.436167  0.504479